## Реестр заблокированных политический ресурсов

Итак, первый шаг, это найти список запрещенки. Роскомнадзор его не предоставляет, зато есть сервис по мониторингу интернета OONI https://explorer.ooni.org/country/RU

Используем API оттуда чтобы получить список запрещенных сайтов.

In [1]:
# Данные
#
# https://api.ooni.io/api/_/website_networks?probe_cc=RU
# http://pravo.minjust.ru/extremist-materials?combine=&page=307

In [2]:
import requests

In [3]:
r = requests.get('https://api.ooni.io/api/_/website_networks?probe_cc=RU')

In [4]:
asn_list = [i['probe_asn'] for i in r.json()['results']]
len(asn_list)

307

Это идентификаторы провайдеров, по которым собирается статистика

In [5]:
asn_list

[12389,
 8402,
 25513,
 48642,
 42387,
 35807,
 42610,
 12714,
 3216,
 8790,
 8334,
 31163,
 21479,
 42668,
 25490,
 31213,
 51604,
 41668,
 44724,
 31257,
 41733,
 15774,
 15640,
 12668,
 8492,
 34533,
 20632,
 8359,
 35000,
 6856,
 42511,
 29194,
 50716,
 16345,
 42682,
 47165,
 31200,
 8615,
 47438,
 20485,
 8427,
 15582,
 25159,
 51547,
 42893,
 8580,
 56981,
 12737,
 50676,
 28890,
 58304,
 56724,
 49342,
 38917,
 34757,
 47655,
 196705,
 13335,
 204297,
 50240,
 35539,
 50923,
 51341,
 15493,
 48149,
 8331,
 24955,
 51659,
 44927,
 25086,
 8570,
 31133,
 42065,
 28891,
 2848,
 28840,
 8369,
 29124,
 39153,
 51813,
 41843,
 51570,
 50473,
 12958,
 43966,
 60192,
 206804,
 41330,
 203730,
 34300,
 43274,
 31224,
 60139,
 199524,
 6939,
 203337,
 12722,
 31364,
 44493,
 39578,
 49392,
 31214,
 50544,
 206873,
 42429,
 209476,
 15378,
 33991,
 49048,
 57128,
 41661,
 198367,
 202422,
 8732,
 28884,
 205569,
 197460,
 29226,
 35723,
 41691,
 29497,
 44587,
 50542,
 44030,
 16083,
 494

In [6]:
def get_asn(asn, limit, offset):
    link = "https://api.ooni.io/api/_/website_urls?probe_cc=RU&probe_asn={}&limit={}&offset={}".format(asn, limit, offset)
    r = requests.get(link)
    return [i for i in r.json()['results']]

In [7]:
from progress.bar import Bar

In [8]:
# links = {}

# for asn in Bar('Countdown', check_tty=False).iter(asn_list):
#     links[asn] = []
#     offset = 0
#     while True:
#         batch = get_asn(asn, 50, offset*50)
#         links[asn] += batch
#         if len(batch) < 50 or offset > 10:
#             break
#         offset += 1
    

Countdown |#                               | 14/307


KeyboardInterrupt: 

In [9]:
import json
# with open('01-ooni-asn-links.json', 'w') as fp:
#     json.dump(links, fp,  indent=4)

f = open('01-ooni-asn-links.json')
links = json.load(f)

In [10]:
list(links.values())[0][0]

{'anomaly_count': 242,
 'confirmed_count': 242,
 'failure_count': 1,
 'input': 'http://khodorkovsky.ru/',
 'total_count': 243}

In [11]:
import itertools
from urllib.parse import urlparse

links_set = set([urlparse(link['input']).netloc for link in itertools.chain(*links.values()) if link['anomaly_count'] > 0])

## Список заблокированных ресурсов по версии OONI

In [12]:
links_set

{'hangouts.google.com',
 'tails.boum.org',
 'blueskyswimwear.com',
 'shamash.org',
 'www.globalpride2020.org',
 'www.hotspotshield.com',
 'jainworld.com',
 'pirateparty.org.au',
 'download.cnet.com',
 'www.03.ru',
 'www.piratpartiet.se',
 'www.inbox.com',
 'www.hrcr.org',
 'www.strana.ru',
 'nordvpn.com',
 'www.wnd.com',
 'storymaps.arcgis.com',
 'team29.org',
 'www.gearthblog.com',
 'www.owl.ru',
 'www.getdrupe.com',
 'hotmail.msn.com',
 'theglobalobservatory.org',
 'www.webwombat.com.au',
 'www.dopplr.com',
 'libgen.li',
 'timesofindia.indiatimes.com',
 'libgen.lc',
 'www.joinclubhouse.com',
 'joinmastodon.org',
 'www.rambler.ru',
 'www.unwomen.org',
 'freespeechdebate.com',
 'lib.rus.ec',
 'stealthchat.com',
 'www.agentura.ru',
 'www.tialsoft.com',
 'www.abortionno.org',
 'www.jesussaves.cc',
 'chaturbate.com',
 'fotostrana.ru',
 'www.sendthisfile.com',
 'sexetc.org',
 'www.namarsh.ru',
 'shtab.navalny.com',
 'www.shram.kiev.ua',
 'www.idrc.ca',
 'www.mysites.ru',
 'mospat.ru',
 'ww

Теперь попробуем для каждого построить временной ряд блокировок

In [13]:
# https://api.ooni.io/api/v1/measurements?limit=50&failure=false&probe_cc=RU&domain=www.deti-404.com&probe_asn=12389&test_name=web_connectivity&since=2020-01-12&until=2022-02-12&anomaly=true&offset=50


def get_blocked_url_data(asn, domain):
    all_items = []
    i = 0
    print(domain)
    while True:
        url = "https://api.ooni.io/api/v1/measurements?limit=50&failure=false&probe_cc=RU&domain={}&probe_asn=12389&test_name=web_connectivity&since=2015-01-01&until=2022-02-12&anomaly=true&offset={}".format(domain, i*50)
        try:
            results = requests.get(url).json()['results']
        except e:
            continue

        all_items += results
        if len(results) < 50:
            break
        i += 1

    return all_items

Соберём данные по провайдеру 12389

In [14]:
data_by_domain = {}

for domain in Bar('BlocksByTime', check_tty=False).iter(links_set):
    data_by_domain[domain] = get_blocked_url_data(12389, domain)

BlocksByTime |                                | 0/1530

hangouts.google.com


BlocksByTime |                                | 1/1530

tails.boum.org


BlocksByTime |                                | 2/1530

blueskyswimwear.com


BlocksByTime |                                | 3/1530

shamash.org


BlocksByTime |                                | 4/1530

www.globalpride2020.org


In [ ]:
with open('02-blocks-by-time.json', 'w') as fp:
    json.dump(data_by_domain, fp,  indent=4)